In [1]:
#Miami, December 27th, 2019
#Renato Kogeyama

# Due bad performance, I am updating the script to more efficient coding

# This script generates a citation ouput with patent|backward citation|cumulated backward citation | year
# The output generates a citation record by year, considering direct and indirect citation


In [2]:
import pandas as pd
import re
import matplotlib.pyplot as plot

import sys
sys.path.append('/home/rkogeyam/scripts/')
from sampler import sampler


In [3]:
%matplotlib inline

In [4]:
fname = r'/home/rkogeyam/PATENT_CITATION/uspatentcitation.tsv'
dst = '/home/rkogeyam/PATENT_CITATION/cit_tree.csv'

In [5]:
%%time
df=pd.read_csv(fname, sep='\t')

# sample_size=100
# df=sampler(fname, sample_size, sep='\t')

CPU times: user 3min 40s, sys: 15 s, total: 3min 55s
Wall time: 3min 55s


In [6]:
df['patent_id']=df['patent_id'].astype(str)

In [7]:
df['citation_id']=df['citation_id'].astype(str)

In [8]:
df.dtypes

uuid           object
patent_id      object
citation_id    object
date           object
name           object
kind           object
country        object
category       object
sequence        int64
dtype: object

In [9]:
# drop rows for which column citation_id has a length smaller than 5
indexNames = df[df['citation_id'].map(len) < 5].index
df.drop(indexNames , inplace=True)

In [10]:
# drop rows for which column patent_id has a length smaller than 5
indexNames = df[df['patent_id'].map(len) < 5].index
df.drop(indexNames , inplace=True)

In [11]:
%%time
#stripping non desired characters but keeping the originals for later check
#could merge unwanted information
cleaning_patent=lambda x:re.sub('([^a-zA-Z0-9]+)', "", x)
df['citation_id']=df['citation_id'].apply(cleaning_patent)
df['patent_id']=df['patent_id'].apply(cleaning_patent)
# df['citation_id'] = df['citation_id'].str.extract('([a-zA-Z0-9]+)', expand=True)
# df['patent_id'] = df['patent_id'].str.extract('([a-zA-Z0-9]+)', expand=True)

CPU times: user 7min 51s, sys: 107 ms, total: 7min 51s
Wall time: 7min 51s


In [12]:
#stripping trailing white spaces
df['patent_id'] = df['patent_id'].str.strip()
df['citation_id'] = df['citation_id'].str.strip()

In [13]:
%%time
#to avoid problems when converting object to datetime format
#replacing day or month would not affect the output
df.date.replace({'-00':'-01'}, regex=True, inplace=True)
#replacing the year could be more problematic
#for now, I am just dropping


CPU times: user 1min 14s, sys: 6 ms, total: 1min 14s
Wall time: 1min 14s


In [14]:
%%time
df['date']=pd.to_datetime(df.date,format='%Y-%m-%d', errors='coerce') #attention to date format - original data is year-month-day
# df['date_format']=pd.to_datetime(df.date,format='%Y-%m-%d', errors='coerce')

CPU times: user 32 s, sys: 900 ms, total: 32.9 s
Wall time: 32.8 s


In [15]:
%%time
#first, i'll run only patents with some kind of citation
#as we have many zero citations in the dataset, this should decrease processing time

cited_patents=df.groupby(['citation_id']).count().iloc[:, 1] #Series, patent-level
# cited_patents=df.groupby(['citation_id']).count() 

CPU times: user 1min 27s, sys: 2.13 s, total: 1min 29s
Wall time: 1min 29s


In [16]:
%%time
cited_patents.dropna(0, inplace=True) #Series

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 18.1 µs


In [17]:
cited_patents.shape

(7049261,)

In [18]:
%%time
cited_patents=cited_patents.reset_index() #Dataframe

CPU times: user 3.48 s, sys: 173 ms, total: 3.65 s
Wall time: 3.65 s


In [19]:
cited_patents.rename(columns={'patent_id': 'back_citation'}, inplace=True)

In [22]:
%%time
# cited_patents.rename(columns={'patent_id': 'back_citation'}, inplace=True)

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 5.01 µs


In [23]:
%%time
#this is the trick calculation
#I match the total of citations received by the citing patent (identified by patent_id)
#In the next step I sum all citations received by the citing patents

df=df.merge(cited_patents, how='inner', left_on='patent_id', right_on='citation_id')
df.rename(columns={'back_citation': 'parent_back_citation'}, inplace=True)

CPU times: user 4min, sys: 4.13 s, total: 4min 4s
Wall time: 4min 4s


In [24]:
df.head()

,uuid,patent_id,citation_id_x,date,name,kind,country,category,sequence,citation_id_y,parent_back_citation
0,000007b7c0x3n9iy1othb9hz7,9009250,8127342,2012-02-01,Boynton et al.,B2,US,cited by applicant,622,9009250,5
1,07tpzlvtmapgex9w34asf96dt,9009250,7593714,2009-09-01,Schultz et al.,B2,US,cited by applicant,548,9009250,5
2,0appyqdtnf0cku8wpgwntz94f,9009250,7177912,2007-02-01,Ignatoff et al.,B1,US,cited by applicant,463,9009250,5
3,0c6h2hmy5e09232xevxw6rrth,9009250,5819172,1998-10-01,"Campana, Jr. et al.",A,US,cited by applicant,111,9009250,5
4,0f2cqtzyd81dcysv7c9ezc2xa,9009250,6757696,2004-06-01,Multer et al.,B2,US,cited by applicant,362,9009250,5


In [25]:
%%time
#NaN in citations means no citation
df.fillna(0, inplace=True)

CPU times: user 1min 38s, sys: 1.45 s, total: 1min 39s
Wall time: 1min 39s


In [26]:
%%time
#now I collapse df to become a patent level dataframe
#citation_id_x is the original cited patent
# patent_level_df=df.groupby('citation_id_x').sum().iloc[:,2]
patent_level_df=df.groupby('citation_id_x').sum()

CPU times: user 1min 28s, sys: 1.78 s, total: 1min 30s
Wall time: 1min 30s


In [28]:
# lastly, i merge the number of citations to patent_level_df
# patent_level has the information about the citing patent

outcome=cited_patents.merge(patent_level_df, how='outer', left_on='citation_id', right_index=True)

In [29]:
outcome.fillna(0, inplace=True)

In [30]:
outcome.head()

,citation_id,back_citation,sequence,parent_back_citation
0,0000000,1,0.0,10.0
1,000004,1,0.0,1.0
2,000006,1,0.0,11.0
3,0000482,1,0.0,30.0
4,0000H94,1,0.0,14.0


In [31]:
outcome.drop(['sequence'], axis=1, inplace=True)

In [32]:
# outcome=outcome.loc[:,['citation_id','back_citation','citation_id_y','parent_back_citation']].set_index('citation_id')

In [33]:
outcome.describe()

,back_citation,parent_back_citation
count,7.049261e+06,7.049261e+06
mean,1.297236e+01,2.194859e+02
std,3.053004e+01,1.498394e+03
min,1.000000e+00,0.000000e+00
25%,2.000000e+00,5.000000e+00
50%,5.000000e+00,2.600000e+01
75%,1.300000e+01,1.080000e+02
max,3.922000e+03,2.274790e+05


In [34]:
outcome.head()

,citation_id,back_citation,parent_back_citation
0,0000000,1,10.0
1,000004,1,1.0
2,000006,1,11.0
3,0000482,1,30.0
4,0000H94,1,14.0


In [35]:
%%time
outcome.to_csv(dst)

CPU times: user 33.1 s, sys: 76 ms, total: 33.2 s
Wall time: 33.3 s
